In [2]:
import pandas as pd
from dotenv import load_dotenv
import os
import mysql.connector
import queries
import plotly.express as px
import plotly.graph_objects as go

In [3]:
load_dotenv()

True

In [4]:
user = "root"
password = os.getenv('db_pw')
host = "127.0.0.1"
port = "3306"
db = "retail_online"

In [5]:
def connect_to_database(host, user, password, database):
    return mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )

def query_to_dataframe(connection, query):
    results = execute_query(connection, query)
    return pd.DataFrame(results)

def execute_query(connection, query):
    cursor = connection.cursor(dictionary=True)
    cursor.execute(query)
    results = cursor.fetchall()
    cursor.close()
    return results

In [6]:
connection = connect_to_database(host, user, password, db)

#### Objective 1

In [7]:
query_1 = queries.return_query_1()
df_1 = query_to_dataframe(connection, query_1)
df_1['country'] = df_1['country'].apply(lambda x: x.capitalize())

In [22]:
fig = px.bar(df_1.sort_values(by="total_revenue", ascending=True), x="total_revenue", y="country", orientation='h',
             labels={'country': 'Country', 'total_revenue': 'Total Revenue (GBP)'})
fig.update_layout(width=800, height=600, xaxis=dict(side='top'))
fig.update_traces(marker_color='#4285f4')
fig.show()

#### Objective 2

In [9]:
query_2 = queries.return_query_2()
df_2 = query_to_dataframe(connection, query_2)
df_2['country'] = df_2['country'].apply(lambda x: x.capitalize())

In [10]:
fig = px.bar(df_2.sort_values(by="item_quantity_sold", ascending=True), x="item_quantity_sold", y="highest_selling_item", orientation='h',
             color='country', labels={'country': 'Country', 'highest_selling_item': 'Item ID', 'item_quantity_sold': 'Quantity Sold'})
fig.update_layout(width=800, height=600, xaxis=dict(side='top'))
fig.show()

#### Objective 3

In [11]:
connection = connect_to_database(host, user, password, db)
query_3 = queries.return_query_3()
df_3 = query_to_dataframe(connection, query_3).sort_values(by="item_quantity_sold", ascending=False)
df_3['total_revenue_local_currency'] = df_3['total_revenue_local_currency'].apply(lambda x: round(x,0))
df_3['unit_price_local_currency'] = df_3['unit_price_local_currency'].apply(lambda x: round(x,2))
df_3['country'] = df_3['country'].apply(lambda x: x.capitalize())

In [21]:
fig = go.Figure(data=[go.Table(columnorder=[1,2], columnwidth=[4, 18],
                 header=dict(values=['<b>Item ID</b>', '<b>Description</b>']),
                 cells=dict(values=[df_3['highest_selling_item'], df_3['description']], align=['center', 'left']))
                     ])
fig.update_layout(width=800, height=600)
fig.show()

In [19]:
fig = go.Figure(data=[go.Table(columnorder=[1,2,3,4,5,6], columnwidth=[50, 120, 30, 40, 40, 40],
              header=dict(values=['<b>Country</b>', '<b>Description</b>', '<b>Price</b>', '<b>Currency</b>', '<b>Quantity Sold</b>', '<b>Total Revenue</b>']),
              cells=dict(values=[df_3['country'], df_3['description'], df_3['unit_price_local_currency'], df_3['currency_name'],
                                 df_3['item_quantity_sold'], df_3['total_revenue_local_currency']],
                                 align=['center', 'left', 'center', 'center', 'center', 'center'],))                  
    ])
fig.update_layout(width=800, height=800)
fig.show()